# Speed Up your ML training with Feature Selection using Swarm Optimizers

What if I told you, you could train your model 1000 times faster and maintain the same accuracy (slightly less or more at times) by simply dropping a few features from your dataset?

Interested? Read on!

## Defining the problem

Imagine you're working with a dataset with 10 features. Using all the 10 features, you're able to make accurate prediction 95% of times.

Now, let's assume your training algorithm has an N^3 complexity. 

Thus, to get a 95% accuracy, you'd be taking 1000 units of time to train.

Assuming that after feature selection, you are able to drop 6 features, and are able to still have a 95% accuracy with just 4 features, your training time comes down to 64 units of time. 


When I do the math, I love just spening 64 units of time (and computational effort) compared to a 1000 units.

tl;dr this is how feature selection works -

![](https://miro.medium.com/max/2400/1*68H8EsCwfqJNxzYdPYtEDw.png)

## Why Swarm Optimizers for Feature Selection?

Because they can solve your problem faster.

Swarm optimizers are a part of the **Heuristic Algorithms** family. A heuristic algorithm is one that is designed to solve a problem in a faster and more efficient fashion than traditional methods by sacrificing optimality, accuracy, precision, or completeness for speed. Heuristic algorithms often times used to solve NP-complete problems, a class of decision problems.

Swarm Intelligence systems employ large numbers of agents interacting locally with one another and the environment. Swarm intelligence refers to the collective behavior of decentralized systems and can be used to describe both natural and artificial systems. Specific algorithms for this class of system include the particle swarm optimization algorithm, the ant colony optimization algorithm, and artificial bee colony algorithm. Each of the previous algorithms was inspired by the natural, self-organized behavior of animals.

In this report, I shall be presenting you with a library of Binary Swarm Optimization algorithms, consisting of - 

* Binary Genetic Algorithm
* Binary Particle Swarm optimization
* Binary Cuckoo Search
* Binary Firefly algorithm
* Binary Bat Algorithm
* Binary Gravitational Search algorithm
* Binary Dragon Fly Algorithm

## Let's dive in!

In this report, we shall be seeing how to use the various swarm optimization algorithms presented in the library to optimize the number of features we use from our dataset. 

Our goal shall be to maximize the test accuracy of the dataset while minimizing the number of features used for training.

### Step 1 - Import the necessary modules

In [1]:
import numpy as np
from sklearn import svm
from sklearn import model_selection as ms
from tqdm import tqdm
from IPython.display import HTML
from warnings import filterwarnings
filterwarnings('ignore')
%matplotlib inline

In [2]:
import binary_optimization as opt

### Step 2 - Import the dataset

In this report we're working with the **Wine Data Set** provided by UCI ML Repository. 

A brief description of the dataset - 

Using chemical analysis determine the origin of wines

![](https://raw.githubusercontent.com/xprilion/random-storage/master/devcloud/Screenshot%202021-02-21%20at%207.44.55%20PM.png)

In [3]:
with open("wine/0.6/wine_train_data_testrate0.6.txt") as f:
    tr_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
    
with open("wine/0.6/wine_test_data_testrate0.6.txt") as f:
    te_d=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])

with open("wine/0.6/wine_train_label_testrate0.6.txt") as f:
    tr_l=np.array([int(data) for data in f.read().splitlines()])

with open("wine/0.6/wine_test_label_testrate0.6.txt") as f:
    te_l=np.array([int(data) for data in f.read().splitlines()])

### Step 3 - Define a scoring function

In [4]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    clf = svm.LinearSVC()
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([svm.LinearSVC().fit(al_data,tr_y).score(al_test_data,te_y) for i in range(4)])

### Step 4 - Create an evaluation object which uses the scoring function for maxmization

In [5]:
class Evaluate:
    def __init__(self):
        self.train_l = tr_l
        self.train_d = tr_d
        self.K = 4
    def evaluate(self,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in self.train_d])
        kf = ms.KFold(n_splits=self.K)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],self.train_l[tr_ix]).score(al_data[te_ix],self.train_l[te_ix])#.predict(al_test_data)
        s/=self.K
        return s
    def check_dimentions(self,dim):
        if dim==None:
            return len(self.train_d[0])
        else:
            return dim

### Step 5 - Let's run it!

In [6]:
results = {}

In [7]:
%%capture
max_iters = 20
number_of_particles = 20
results["BGA"] = opt.BGA(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BPSO"] = opt.BPSO(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BCS"] = opt.BCS(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BFFA"] = opt.BFFA(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BBA"] = opt.BBA(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BGSA"] = opt.BGSA(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)
results["BDFA"] = opt.BDFA(Eval_Func=Evaluate,n=number_of_particles,m_i=max_iters)

In [8]:
print("Algorithm:\n\t{0}         {1}      {2}      {3}".format("best_features","best_val","number_of_1s", "test_score"))

for key in results.keys():
    s, g, l, a = results[key]
    print("{0}:\n\t{1}          {2:.6f}           {3}             {4:.6f}".format(str(key), "".join(map(str,g)),s,l, test_score(g,tr_d,tr_l,te_d,te_l)))

Algorithm:
	best_features         best_val      number_of_1s      test_score
BGA:
	1111000101010          0.930556           7             0.883178
BPSO:
	0001011111000          0.944444           6             0.894860
BCS:
	1001001111000          0.944444           6             0.894860
BFFA:
	0000001101100          0.958333           4             0.878505
BBA:
	0010010111110          0.930556           7             0.859813
BGSA:
	1001011110110          0.928922           8             0.878505
BDFA:
	1111101011110          0.944444           10             0.866822


### Let's see some cool animations!

In the following animations, the top 10 generations/particles of the swarm are represented. Each column denotes a feature of the dataset, and red dots in each row denote the inclusion of that feature for that generation/particle.

The top 10 particles are ranked in descending order of their score on the evaluation function and displayed.

Notice how the swarm's top 10 particles change over time to yield the result.

#### Binary Geneting Algorithm

In [16]:
HTML(results["BGA"][3].to_html5_video())

#### Binary Particle Swarm Optimization

In [10]:
HTML(results["BPSO"][3].to_html5_video())

#### Binary Cuckoo Search

In [11]:
HTML(results["BCS"][3].to_html5_video())

#### Binary Firefly Algorithm

In [12]:
HTML(results["BFFA"][3].to_html5_video())

#### Binary Bat Algorithm

In [13]:
HTML(results["BBA"][3].to_html5_video())

#### Binary Gravitational Search Algorithm

In [14]:
HTML(results["BGSA"][3].to_html5_video())

#### Binary Dragon Fly Algorithm

In [15]:
HTML(results["BDFA"][3].to_html5_video())

## Conclusion

With the help of this binary swarm optimization library, one can very rapidly determine the minimal subset of features needed to train his/her model in a manner that's time and cost efficient while not compromising much on the accuracy of the model.

These algorithms can be put to several other uses such as determining the answers to equations with unknown variables, etc.

A report by - Anubhav Singh (xprilion@gmail.com)